# Regression with Keras Wide & Deep neural network
In this project we will build an regressor using the Wide & Deep neural network functional API of Keras on california housing dataset. For the regression with a sequential API, see project "A Simple Regression MLP with Keras Sequential API"

In [1]:
import tensorflow as tf
from tensorflow.keras import layers
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split

In [2]:
housing = fetch_california_housing()

X_train_full, X_test, y_train_full, y_test = train_test_split(
    housing.data, housing.target, random_state=369)
X_train, X_valid, y_train, y_valid = train_test_split(
    X_train_full, y_train_full, random_state=369)

In [3]:
tf.random.set_seed(369)

In [4]:
#create all the layers we need
normalization_layer = layers.Normalization()
hidden_layer1 = layers.Dense(30, activation='relu')
hidden_layer2 = layers.Dense(30, activation='relu')
concat_layer = layers.Concatenate()
output_layer = layers.Dense(1)

#construct model by using the layers as functions
input_ = layers.Input(shape=X_train.shape[1:])
normalized = normalization_layer(input_)
hidden1 = hidden_layer1(normalized)
hidden2 = hidden_layer2(hidden1)
concat = concat_layer([normalized, hidden2]) #concatenate the input and the second hidden layer’s output
output = output_layer(concat)

#specifying which inputs and outputs to use
model = tf.keras.Model(inputs=[input_], outputs=[output])
model.summary()

Metal device set to: Apple M1


2023-01-07 09:09:47.179363: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-01-07 09:09:47.179660: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 8)]          0           []                               
                                                                                                  
 normalization (Normalization)  (None, 8)            17          ['input_1[0][0]']                
                                                                                                  
 dense (Dense)                  (None, 30)           270         ['normalization[0][0]']          
                                                                                                  
 dense_1 (Dense)                (None, 30)           930         ['dense[0][0]']                  
                                                                                              

In [5]:
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
model.compile(loss='mse', optimizer=optimizer, metrics=['RootMeanSquaredError'])
normalization_layer.adapt(X_train)
history = model.fit(X_train, y_train, epochs=20, validation_data=[X_valid, y_valid])

mse_test, rmse_test = model.evaluate(X_test, y_test)

2023-01-07 09:09:47.586198: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-01-07 09:09:47.629286: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


Epoch 1/20


2023-01-07 09:09:48.705386: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


363/363 [==============================] - ETA: 0s - loss: 1.3553 - root_mean_squared_error: 1.1642

2023-01-07 09:09:51.721254: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


363/363 [==============================] - 4s 7ms/step - loss: 1.3553 - root_mean_squared_error: 1.1642 - val_loss: 0.5482 - val_root_mean_squared_error: 0.7404
Epoch 2/20
363/363 [==============================] - 2s 6ms/step - loss: 0.4996 - root_mean_squared_error: 0.7068 - val_loss: 0.4514 - val_root_mean_squared_error: 0.6719
Epoch 3/20
363/363 [==============================] - 2s 6ms/step - loss: 0.4240 - root_mean_squared_error: 0.6512 - val_loss: 0.4124 - val_root_mean_squared_error: 0.6422
Epoch 4/20
363/363 [==============================] - 2s 6ms/step - loss: 0.3938 - root_mean_squared_error: 0.6276 - val_loss: 0.3891 - val_root_mean_squared_error: 0.6238
Epoch 5/20
363/363 [==============================] - 2s 6ms/step - loss: 0.3812 - root_mean_squared_error: 0.6174 - val_loss: 0.3788 - val_root_mean_squared_error: 0.6155
Epoch 6/20
363/363 [==============================] - 2s 6ms/step - loss: 0.3819 - root_mean_squared_error: 0.6180 - val_loss: 0.3751 - val_root_mean_s

In [6]:
print(mse_test, rmse_test)

0.3228816092014313 0.5682267546653748


Next, we send 5 features through the wide path (features 0 to 4), and 6 through the deep path (features 2 to 7).

In [7]:
#two input layers
input_wide = layers.Input(shape=[5])
input_deep = layers.Input(shape=[6])
norm_layer_wide = layers.Normalization()
norm_layer_deep = layers.Normalization()

norm_wide = norm_layer_wide(input_wide)
norm_deep = norm_layer_deep(input_deep)

hidden1 = layers.Dense(30, activation="relu")(norm_deep)
hidden2 = layers.Dense(30, activation="relu")(hidden1)
concat = layers.Concatenate()([norm_wide, hidden2])
output = layers.Dense(1)(concat)

model = tf.keras.Model(inputs=[input_wide, input_deep], outputs=[output])
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 6)]          0           []                               
                                                                                                  
 normalization_2 (Normalization  (None, 6)           13          ['input_3[0][0]']                
 )                                                                                                
                                                                                                  
 input_2 (InputLayer)           [(None, 5)]          0           []                               
                                                                                                  
 dense_3 (Dense)                (None, 30)           210         ['normalization_2[0][0]']  

In [8]:
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
model.compile(loss="mse", optimizer=optimizer, metrics=["RootMeanSquaredError"])

X_train_wide, X_train_deep = X_train[:, :5], X_train[:, 2:]
X_valid_wide, X_valid_deep = X_valid[:, :5], X_valid[:, 2:]
X_test_wide, X_test_deep = X_test[:, :5], X_test[:, 2:]

norm_layer_wide.adapt(X_train_wide)
norm_layer_deep.adapt(X_train_deep)

history = model.fit((X_train_wide, X_train_deep), y_train, epochs=20, 
                    validation_data=((X_valid_wide, X_valid_deep), y_valid))

mse_test, rmse_test = model.evaluate((X_test_wide, X_test_deep), y_test)

2023-01-07 09:10:36.854319: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-01-07 09:10:37.541578: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


Epoch 1/20
  1/363 [..............................] - ETA: 1:46 - loss: 8.3002 - root_mean_squared_error: 2.8810

2023-01-07 09:10:38.330132: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


363/363 [==============================] - ETA: 0s - loss: 1.7587 - root_mean_squared_error: 1.3262

2023-01-07 09:10:40.633115: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


363/363 [==============================] - 3s 7ms/step - loss: 1.7587 - root_mean_squared_error: 1.3262 - val_loss: 0.7607 - val_root_mean_squared_error: 0.8722
Epoch 2/20
363/363 [==============================] - 2s 6ms/step - loss: 0.6365 - root_mean_squared_error: 0.7978 - val_loss: 0.5672 - val_root_mean_squared_error: 0.7532
Epoch 3/20
363/363 [==============================] - 2s 7ms/step - loss: 0.5167 - root_mean_squared_error: 0.7188 - val_loss: 0.4812 - val_root_mean_squared_error: 0.6937
Epoch 4/20
363/363 [==============================] - 2s 7ms/step - loss: 0.4560 - root_mean_squared_error: 0.6753 - val_loss: 0.4425 - val_root_mean_squared_error: 0.6652
Epoch 5/20
363/363 [==============================] - 2s 7ms/step - loss: 0.4276 - root_mean_squared_error: 0.6539 - val_loss: 0.4190 - val_root_mean_squared_error: 0.6473
Epoch 6/20
363/363 [==============================] - 2s 6ms/step - loss: 0.4032 - root_mean_squared_error: 0.6350 - val_loss: 0.4010 - val_root_mean_s

In [9]:
print(mse_test, rmse_test)

0.3260728120803833 0.5710278749465942


Let's build a model with 2 inputs and 2 outputs layers:

In [10]:
tf.keras.backend.clear_session()
tf.random.set_seed(369)

In [11]:
input_wide = layers.Input(shape=[5])
input_deep = layers.Input(shape=[6])
norm_layer_wide = layers.Normalization()
norm_layer_deep = layers.Normalization()
norm_wide = norm_layer_wide(input_wide)
norm_deep = norm_layer_deep(input_deep)
hidden1 = layers.Dense(30, activation='relu')(norm_deep)
hidden2 = layers.Dense(30, activation='relu')(hidden1)
concat = layers.concatenate([norm_wide, hidden2])
output = layers.Dense(1)(concat)
aux_output = layers.Dense(1)(hidden2)

model = tf.keras.Model(inputs=[input_wide, input_deep], outputs=[output, aux_output])

In [12]:
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
model.compile(loss=('mse', 'mse'), loss_weights=(0.9, 0.1), optimizer=optimizer, metrics=['RootMeanSquaredError'])

In [13]:
X_train_wide, X_train_deep = X_train[:, :5], X_train[:, 2:]
X_valid_wide, X_valid_deep = X_valid[:, :5], X_valid[:, 2:]
X_test_wide, X_test_deep = X_test[:, :5], X_test[:, 2:]

In [14]:
norm_layer_wide.adapt(X_train_wide)
norm_layer_deep.adapt(X_train_deep)

history = model.fit(
    (X_train_wide, X_train_deep), (y_train, y_train), epochs=20,
    validation_data=((X_valid_wide, X_valid_deep), (y_valid, y_valid))
)

2023-01-07 09:11:26.601006: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-01-07 09:11:27.277895: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


Epoch 1/20
  1/363 [..............................] - ETA: 2:06 - loss: 6.6686 - dense_2_loss: 6.6612 - dense_3_loss: 6.7347 - dense_2_root_mean_squared_error: 2.5809 - dense_3_root_mean_squared_error: 2.5951

2023-01-07 09:11:28.156742: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


363/363 [==============================] - ETA: 0s - loss: 1.4539 - dense_2_loss: 1.3719 - dense_3_loss: 2.1918 - dense_2_root_mean_squared_error: 1.1713 - dense_3_root_mean_squared_error: 1.4805

2023-01-07 09:11:31.391305: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


363/363 [==============================] - 4s 10ms/step - loss: 1.4539 - dense_2_loss: 1.3719 - dense_3_loss: 2.1918 - dense_2_root_mean_squared_error: 1.1713 - dense_3_root_mean_squared_error: 1.4805 - val_loss: 0.7240 - val_dense_2_loss: 0.6797 - val_dense_3_loss: 1.1228 - val_dense_2_root_mean_squared_error: 0.8244 - val_dense_3_root_mean_squared_error: 1.0596
Epoch 2/20
363/363 [==============================] - 3s 10ms/step - loss: 0.6107 - dense_2_loss: 0.5786 - dense_3_loss: 0.9000 - dense_2_root_mean_squared_error: 0.7606 - dense_3_root_mean_squared_error: 0.9487 - val_loss: 0.5615 - val_dense_2_loss: 0.5408 - val_dense_3_loss: 0.7477 - val_dense_2_root_mean_squared_error: 0.7354 - val_dense_3_root_mean_squared_error: 0.8647
Epoch 3/20
363/363 [==============================] - 4s 10ms/step - loss: 0.4901 - dense_2_loss: 0.4700 - dense_3_loss: 0.6716 - dense_2_root_mean_squared_error: 0.6855 - dense_3_root_mean_squared_error: 0.8195 - val_loss: 0.4590 - val_dense_2_loss: 0.4417

In [15]:
weighted_sum_of_losses, main_loss, aux_loss, main_rmse, aux_rmse = \
model.evaluate((X_test_wide, X_test_deep), (y_test, y_test))

162/162 [==============================] - 1s 6ms/step - loss: 0.3308 - dense_2_loss: 0.3234 - dense_3_loss: 0.3973 - dense_2_root_mean_squared_error: 0.5687 - dense_3_root_mean_squared_error: 0.6303


In [16]:
print(weighted_sum_of_losses, main_loss, aux_loss, main_rmse, aux_rmse)

0.3308311104774475 0.3234446048736572 0.39730948209762573 0.5687219500541687 0.6303249001502991


## Build Dynamic Models

In [55]:
#subclassing the Model class
class WideAndDeepModel(tf.keras.Model):
    def __init__(self, units=30, activation='relu', **kwargs):
        super().__init__(**kwargs)
        self.norm_layer_wide = layers.Normalization()
        self.norm_layer_deep = layers.Normalization()
        self.hidden1 = layers.Dense(units, activation=activation)
        self.hidden2 = layers.Dense(units, activation=activation)
        self.main_output = layers.Dense(1)
        self.aux_output = layers.Dense(1)
    
    def call(self, inputs):
        input_wide, input_deep = inputs
        norm_wide = self.norm_layer_wide(input_wide)
        norm_deep = self.norm_layer_deep(input_deep)
        hidden1 = self.hidden1(norm_deep)
        hidden2 = self.hidden2(hidden1)
        concat = layers.concatenate([norm_wide, hidden2])
        output = self.main_output(concat)
        aux_output = self.aux_output(hidden2)
        return output, aux_output
    
tf.random.set_seed(369)
model = WideAndDeepModel(30, activation='relu', name='dynamic_model')

In [56]:
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
model.compile(loss='mse', loss_weights=[0.9, 0.1], optimizer=optimizer, metrics=['RootMeanSquaredError'])
model.norm_layer_wide.adapt(X_train_wide)
model.norm_layer_deep.adapt(X_train_deep)

history = model.fit(
    (X_train_wide, X_train_deep), (y_train, y_train), epochs=10,
    validation_data=((X_valid_wide, X_valid_deep), (y_valid, y_valid)))

weighted_sum_of_losses, main_loss, aux_loss, main_rmse, aux_rmse = \
model.evaluate((X_test_wide, X_test_deep), (y_test, y_test))

2023-01-07 11:03:58.069625: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-01-07 11:03:58.083944: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-01-07 11:03:58.783878: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-01-07 11:03:58.797908: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


Epoch 1/10
  6/363 [..............................] - ETA: 3s - loss: 5.7931 - output_1_loss: 5.7476 - output_2_loss: 6.2028 - output_1_root_mean_squared_error: 2.3974 - output_2_root_mean_squared_error: 2.4905  

2023-01-07 11:03:59.700319: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


363/363 [==============================] - ETA: 0s - loss: 1.4539 - output_1_loss: 1.3719 - output_2_loss: 2.1918 - output_1_root_mean_squared_error: 1.1713 - output_2_root_mean_squared_error: 1.4805

2023-01-07 11:04:02.978978: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


363/363 [==============================] - 4s 10ms/step - loss: 1.4539 - output_1_loss: 1.3719 - output_2_loss: 2.1918 - output_1_root_mean_squared_error: 1.1713 - output_2_root_mean_squared_error: 1.4805 - val_loss: 0.7240 - val_output_1_loss: 0.6797 - val_output_2_loss: 1.1228 - val_output_1_root_mean_squared_error: 0.8244 - val_output_2_root_mean_squared_error: 1.0596
Epoch 2/10
363/363 [==============================] - 4s 10ms/step - loss: 0.6107 - output_1_loss: 0.5786 - output_2_loss: 0.9000 - output_1_root_mean_squared_error: 0.7606 - output_2_root_mean_squared_error: 0.9487 - val_loss: 0.5615 - val_output_1_loss: 0.5408 - val_output_2_loss: 0.7477 - val_output_1_root_mean_squared_error: 0.7354 - val_output_2_root_mean_squared_error: 0.8647
Epoch 3/10
363/363 [==============================] - 4s 10ms/step - loss: 0.4901 - output_1_loss: 0.4700 - output_2_loss: 0.6716 - output_1_root_mean_squared_error: 0.6855 - output_2_root_mean_squared_error: 0.8195 - val_loss: 0.4590 - val_

In [57]:
print(weighted_sum_of_losses, main_loss, aux_loss, main_rmse, aux_rmse)

0.35186731815338135 0.34291303157806396 0.43245503306388855 0.585587739944458 0.6576131582260132
